In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
def simulate_gbm(Total_paths, steps_per_year, T, S0, mu, sigma):

    dt = 1/steps_per_year
    dW = np.random.normal(0, np.sqrt(dt),size=(Total_paths,steps_per_year*T)).T
    St = np.exp((mu-sigma ** 2 / 2) * dt + sigma * dW)
    St = np.vstack([np.ones(Total_paths),St])
    St[0] = S0
    St = St.cumprod(axis=0)
    return St

In [4]:
# Parameter
# number of exercise dates per year
M = 50
# total number of paths
N = 100000
# number of iterations
n = 100
# time in years
T = 1
# initial stock price
S0 = 36
# volatility
sigma = 0.2
# strike
strike = 40
# let the annual risk-free interest rate
r = 0.06


# discounted price for all iterations
Price = [0]*n
# path info from iteration 1 to current iteration - a dictionary with k as the key
U = {k: np.zeros([3, 3]) for k in range(1, T*M + 1)}
V = {k: np.zeros([3,]) for k in range(1, T*M + 1)}

# model info for each time step - a dictionary with k as the key
alphas = dict()


# initializaiton:
paths = simulate_gbm(int(N/n), M, T, S0, r, sigma)
euro_discounted_prices = np.ones_like(paths) * np.exp(-r/M)
final_payoff = strike - paths[-1]
euro_discounted_prices[0] = np.where(final_payoff > 0, final_payoff, 0)
euro_discounted_prices = euro_discounted_prices.cumprod(axis=0)
euro_discounted_prices = euro_discounted_prices[::-1]
for k in range(1, T*M+1):
    x1 = paths[k]
    x2 = x1*x1

    xs = np.column_stack([x1,x2])

    if k==T*M:
        y = euro_discounted_prices[k]
    else:
        y = euro_discounted_prices[k+1] * np.exp(-r/M)
    model_sklearn = LinearRegression()
    model = model_sklearn.fit(xs, y)
    alphas[k] = np.append(np.array([model.intercept_]),model.coef_)


for i in range(1, n+1):
    # for one interation
    # step 1: paths simulation
    paths =simulate_gbm(int(N/n), M, T, S0, r, sigma)

    # step 2.1: payoff value calculation
    payoff_values = strike - paths
    payoff_values[payoff_values<0] = 0
    # step 2.2: continuation value estimate:
    continuation_values = np.zeros_like(paths)
    for k in range(1, T*M+1):
        # in each time step, calculate the continuation values for different paths
        # using corresponding regression model(alpha_k)
        x0 = np.ones_like(paths[k]) # intercept
        x1 = paths[k]
        x2 = x1 * x1
        xs = np.column_stack([x0, x1, x2])
        continuation_values[k]= xs.dot(alphas[k])

    # step 3: comparison between continuation value and exrecise value
    discounted_price = payoff_values
    for k in range(T*M-1, 0, -1):
        no_exercise = discounted_price[k] < continuation_values[k]
        discounted_price[k][no_exercise] = discounted_price[k+1][no_exercise] * np.exp(-r/M)

    # step 4: sum of t=1 prices
    Price[i-1]= np.exp(-r/M) * discounted_price[1].sum()

    # step 5:update the regression models
    for k in range(1, T*M+1):
        x0 = np.ones_like(paths[k]) # intercept
        x1 = paths[k]
        x2 = x1 * x1
        f_T = np.column_stack([x0, x1, x2])
        f = f_T.T
        if k == T*M:
            y = discounted_price[k]
        else:
            y = discounted_price[k + 1] * np.exp(-r / M)

        u = f.dot(f_T)
        v = f.dot(y)

        U[k] = U[k] + u
        V[k] = V[k] + v

        reg = np.linalg.lstsq(U[k], V[k], rcond=None)
        alphas[k] = reg[0]


# weights given to different iterations
W = np.array([0.99*(i-1) for i in range(1,n+1)])
W = np.tanh(W)
W = 1 - 0.5 * (1 - W)

Price = np.array(Price) / (N/n)
Option_price = sum(Price * W) / W.sum()

print(Option_price, Price)

4.4015537172268395 [4.1070147  4.293577   4.38969587 4.36360264 4.4044007  4.36566387
 4.41176493 4.41114441 4.44006601 4.33973433 4.32049824 4.41036772
 4.33957108 4.38731331 4.50767266 4.52381392 4.40376813 4.37731619
 4.26216639 4.56173582 4.43324216 4.48258892 4.48177257 4.24760796
 4.46708407 4.45216448 4.36319295 4.28011499 4.42944917 4.44266855
 4.43076267 4.23337737 4.42737331 4.41193694 4.35591679 4.40866095
 4.33712336 4.28599658 4.47820321 4.33776736 4.37874335 4.29983356
 4.23187986 4.48043858 4.45245363 4.25355227 4.63270444 4.2931277
 4.46766941 4.4216311  4.38719815 4.54983242 4.44881176 4.44202803
 4.18220999 4.35902451 4.40033531 4.56126267 4.42730534 4.44649045
 4.33299412 4.31532346 4.42929111 4.30262197 4.33503436 4.39170768
 4.46417994 4.40660327 4.37480698 4.59629567 4.29687901 4.33208009
 4.40746668 4.39626916 4.47938197 4.43030102 4.33932893 4.36437817
 4.49857367 4.36742358 4.39198925 4.42517782 4.56805633 4.49279633
 4.48274009 4.3716902  4.50796403 4.28010129